<a href="https://colab.research.google.com/github/Miseq/EllipticCurves/blob/master/krzywe_eliptyczne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacja bibliotek i importowanie ich

Poniższe opracowanie do poprawnego działania wymaga zainstalowania kilku potrzebnych pakietów, oraz zaimportowanie. Uruchomienie poniższych fragmentów kodu wykona te operacje:


In [0]:
# instalowanie bibliotek wymaganych do działania kodu
# zapobiega outputowi na parę stron
%%capture 
!apt-get install python-dev libgmp3-dev
!pip install ecpy
!pip install fastecdsa
!pip install eciespy
!pip install tinyec
!pip install python_secrets
!pip install pycryptodome

print("Done!")

In [0]:
# importowanie wcześniej zainstalowanych bibliotek
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ecies.utils import generate_eth_key, generate_key
from ecies import encrypt, decrypt
from fastecdsa import curve, ecdsa, keys
from hashlib import sha384
from tinyec import registry
import tinyec.ec as ec
from Crypto.Cipher import AES
import hashlib, secrets, binascii
from ipywidgets import interactive
import warnings

print("Done!")

Done!


# **Krzywe eliptyczne - wyjaśnienie**


Krzywe eliptyczne są rodzajem gładkich krzywych algebraicznych, które znalazły szerokie zastosowanie we współczesnej kryptografii. Swoją popularność zawdzięczają matematycznie łatwemu generowaniu kluczy oraz dużej złożoności obliczeniowej, często nieopłacalnej czasowo, wymaganej do złamania szyfru stworzonego przez daną krzywą.



# **Wymiana kluczy algorytmem Diffie-Hellman**


Aby zrozumieć działanie oraz znaczenie krzywych eliptycznych w krytpografii najpierw *należy* wyjaśnić koncept szyfrowania wiadomości za pomocą kluczy publicznych oraz prywatnych, następnie zaś podstawową metodę wymiany Diffie-Hellmana.


Jako przykładowy problem wyobraźmy sobie dwóch użytkowników, Adam oraz Bartek, którzy chcą porozumiewać się przez internet prywatnie. Jeden użytkownik wysyła więc do drugiego dowolną cyfrę **$G$**, będącą kluczem publicznym następnie zaś każdy użytkownik tworzy własny klucz prywatny.
- Adam: **$a$**
- Bartek: **$b$**

Kolejnym krokiem jest podniesienie klucza publicznego (**$G$**) do potęgi klucza prywatnego, tworząc w ten sposób zmienną, którą nazwiemy **H**:
- Adam: **$G^a = Ha$**
- Bartek: **$G^b = Hb$**

Następnie wysyłają sobie nawzajem zmienne **$H$**, przez co Adam posiada **$Hb$** zaś Bartek **$Ha$**. W tym miejscu oboje znów podnoszą przesłaną liczbę do potęgi klucza prywatnego, dzięki czemu zostaje wykorzystana własność potęgowania sprowadzająca się do równania:
- **$Ha^b == Hb^a$** gdyż **$((H)^a)^b = H^{ab}$** 

Obaj użytkownicy otrzymują w wyniku tą samą liczbę **S**, która jest potwierdzeniem iż wymiania wiadomości zachodzi (teoretycznie) pomiędzy poprawnymi użytkownikami.

![alt text](https://drive.google.com/uc?id=1K4bwZ8vbT9bz_8VSOCMc2h2wZT1wn8Qw)



## **Problem z przechwyceniem dwóch składowych równania**

W krytycznej fazie wymiany zarówno **$G$** jak i **$Ha$** wysyłane sa publicznie przez internet, jeśli ktokolwiek przechwyciłby te wiadomości, odkrycie wartości prywatnego klucza **$a$** nie sprawiałoby problemu i sprowadzałoby się (w uproszczeniu) do poniższego działania:
\begin{equation}
  2^n = 16, n = ?
\end{equation}

Rozwiązaniem tego problemu jest wprowadzenie operacji **modulo**. Dzięki jej wprowadzeniu nie można mieć pewności do jakiej rzeczywistej wartości odnosi się dana liczba, z cyklicznej natury działania operacji **mod**.

\begin{equation}
  2^n\ mod\ 17 = 16, n = ?
\end{equation}

Po zastosowaniu tej operacji, nawet jeśli atakujący przechwyci dwie wartości, dalej nie ma pewności czy **n** będzie równe 4, 12, 20, 36 czy 2988 ponieważ wszystkie te liczby spełniają równanie.

## **Wykorzystanie krzywych eliptycznych do usprawnienia protokołu wymiany kluczy**

Problemem z standardowym protokołem wymiany kluczy polega na ich dużych rozmiarach. W ostatnich latach, z podu gwałtownego wzrostu popularności Internetu Rzeczy (IoT), zwiększa się liczba coraz mniejszych urządzeń potrzebujących bezpiecznego połączenia z internetem. Z powodu ograniczeń mocy obliczeniowej, urządzenia te nie są w stanie generować odpowiednio skomplikowanych kluczy do zapewnienia bezpiecznego transferu. Jednocześnie należy tworzyć coraz bardziej skomplikowane klucze, które zapewnią bezpieczeństwo w nieustannej wojnie z hackerami.


Całościowo problem można sprowadzić do zdania:
**Klucze muszą być coraz mniejsze a zarazem klucze muszą być coraz większe**.

Rozwiązaniem tego problemu okazało się zastosowanie krzywych eliptycznych jako sposobu na generowanie kluczy. Otóż klucze wygenerwaone w ten sposób, potrafią mieć dzisięciokrotnie mniejszy rozmiar (w bitach) będąc jednocześnie bardziej skomplikowane do odgadnięcia przez stronę trzecią.

# Krzywa eliptyczna



Podstawowa krzywa eliptyczna jest wynikiem działania funkcji  **$y^2=x^3+ax+b$**: 

![alt text](https://drive.google.com/uc?id=1PktWCdIfCjsNX2KOT5GFNIKUJyuD_Tej)





Gdzie zmienna **$a$** odpowiada za wybrzuszenie, gdzie wartości poniżej $-1$ 'odrywają' wybrzuszenie od krzywej, zaś wartości powyżej stopniowo wypłaszczają je.
Zmienna **$b$** odpowiada za szerokość wybrzuszenia. Zauważyć to można na rysunku przedstawionym poniżej:

![alt text](https://drive.google.com/uc?id=1pckQ3cAuAjGamWlR8toSXelj3ZUwHeT2)

## Rysowanie krzywych eliptycznych przy pomocy języka Python

Aby ułatwić rysowanie krzywych, najpierw przeprowadzono obliczenia w ciele liczb rzeczywistych. Następnie przeprowadzono odwzorowanie konturowe na cało liczb rzeczywistych.

In [0]:
y, x = np.ogrid[-5:5:100j, -5:5:100j] # Tworzenie pola na którym zostanie wyrysowana krzywa

# Funkcja rysująca
def g(a, b):
    # Dodatkowa funkcja pomocnicza, która pomaga formatować tytuł wykresu
    def sign_format(a):
      return "- %g" % (a*(-1)) if a < 0 else "+ %g" % a 
    
    # Wzór funkcji w ciele liczb zespolonych
    basic_curve = pow(y, 2) - pow(x, 3) - x * a - b

    # Odbicie wykresu w ciele liczb zespolonych na liczby rzeczywiste
    plt.contour(x.ravel(), y.ravel(), basic_curve, [0])
    plt.grid() # Włączenie siatki
  
    # Ustawienie tytułu
    plt.title(f"$y^2 = x^3 {sign_format(a)} \cdot x {sign_format(b)} $")
    plt.show()

# Uruchomienie interaktywnego wykresu, z suwakami dla wartości a i b.
# Zakres wartości zmiennych od -5 do 5, a krok ustawień to 0.1
l = interactive(g, a=(-5, 5, 0.1), b=(-5, 5, 0.1))

display(l) # Wyświelenie wykresu.


interactive(children=(FloatSlider(value=0.0, description='a', max=5.0, min=-5.0), FloatSlider(value=0.0, descr…

Powyższy przykład jest prosty a zatem niezbyt bezpieczny. Bardziej zaawansowane krzywe mogą mieć wzór na przykład:
$y^2 = x^3-3x+19596161053329239268181228455226581162286252326261019516900162717091837027531392576647644262320816848087868142547438$

# **Wyznaczanie kolejnych n-tych punktów na krzywej eliptycznej**

Przykładowy widok krzywej eliptycznej z dwolonie wybranym punktem 'g'
.

![Przykładowa krzywa eliptyczna](https://drive.google.com/uc?id=1wU5Cm7jgz4tqLkFeC8zcGhxSqJCLiZ4c)

Aby wyznaczyć punkt '2g' przeprowadzamy styczną w punkcie 'g', następnie zaś odbijamy symetrycznie względem osi X.

![Wyrpowadzenie tangensa w celu wyznaczenia punktu 2g](https://drive.google.com/uc?id=1UYLT3CI8R2K53rNRYwPec1oe8IgCPu90)

Do wyznaczenia punktu 3g(tutaj wyróżnione 'X') przeprowadzamy prostą przechodzącą przez punkt g oraz 2g, a następnie odwracamy wzdłuż osi X miejsce, w którym prosta przecina krzywą eliptyczną.


![alt text](https://drive.google.com/uc?id=1bNPJHwxcqoQ1nolcyZG6_cfT1m8eEeVR)

Schemat powtarzamy do wyznaczania dowlonej ilości następnych punktów Ng na krzywej eliptycznej, gdzie N należy do zbioru liczb rzeczywistych.

![alt text](https://drive.google.com/uc?id=1iZ8N7EnaN9M5qYG93TRoshSxAzR7EVzB)

Ważną zaletą krzywych eliptycznych jest fakt, iż wykreslając prostą przez dwa dowolne punkty, zawsze przetniemy funkcję w maksymalnie jednym miejscu, co sprawia iż nigdy nie zostaniemy postawieni przed wyborem w którym miejscu powinna znaleźć się następna wielokrotnośc wykreślanego punktu.

Łatwo zauważyć, iż wyznaczając kolejne wielokrotności dowolnego punktu g, poruszamy się po danej krzywej eliptycznej w sposób wyglądający na losowy.
Ten sposób poruszania się sprawia iż 

# **Klucze - koncepcyjnie**

Klucz prywatny służy do szyfrowania i deszyfrowania wiadomości pomiędzy użytkownikami. Zazwyczaj jest on wartością określającą, którą wielokrotnością danego punktu **g** jest przedstawiony punkt.
Na przykładzie poniżej mamy dany punkt g, oraz drugi punkt określany jako **?g**, gdzie **?** oznacza pewną wielokrotność punktu g. Może być to zarówno 5g jak i 10g jak i 1000g.

Z powodu charakterystyki budowy krzywych eliptycznych przedstawionych powyżej, poprawne odgadnięcie jaką wartość ma zmienna N, jest niezwykle czasochłonne i wymaga wielu obliczeń.


Dla większych krzywych jest to niemal niemożliwe i tak skomplikowane, że nieopłacalne.



![alt text](https://drive.google.com/uc?id=1yheByQpjiKpSFupjjQJ4eeocXGES6E4i)

## Krzywe eliptyczne na skończonej płaszczyźnie

Przy wyznaczaniu kolejnych punktów na krzywych eliptycznych stosowanych w kryptografii, często stosuje się ograniczanie płaszczyzny krzywej (która normalnie ma nieskończony rozmiar). Dzięki temu, algorymty mogą wyznaczyć współrzędne kolejnych punktów danej krzywej. Stosuje się do tego algorytm [Schoof'a](https://en.wikipedia.org/wiki/Schoof%27s_algorithm). Znajomość ilości punktów na krzywej eliptycznej w ograniczonej płaszczyźnie jest ważna podczas oceniania poziomu skomplikowania i bezpieczeństwa danej krzywej.

Należy pamiętać, iż na płaszczyznach poniżej zaznaczone są punkty afiniczne, zaś sama płaszczyzna nazywana jest płaszczyzną [afiniczną](https://en.wikipedia.org/wiki/Affine_space) służącą do operacji na wektorach i punktach. Mimo iż gołym okiem można zobaczyć symetrię oraz wzory punktów, nie są one reprezentantami krzywej na płaszczyźnie euklidesowej. 

Upraszczając, takie punkty afiniczne przez krzywą wyznaczane są stosując arytmetykę modularną. Następuje zatem utworzenie płaszczyzny, w której oś x oraz y "zawijają się" - podobnie jak to jest w przypadku wartości zmiennych w siatce Karnaugha. Miejsce, w którym następuje zawinięcie, określa modulnik operacji, który jest stały dla całej krzywej i nazywa się polem.

Poniżej zaprezentowano przykład takiej płaszczyzny. Należy jednak zaznaczyć, że współrzędne tych punktów należą do liczb całkowitych.

In [0]:
y, x = np.ogrid[-5:5:100j, -5:5:100j]

# Funkcja wspomagająca formatowanie
def sign_format(a):
    return "- %g" % (a*(-1)) if a < 0 else "+ %g" % a 

# Funkcja rysująca krzywą w przestrzeni euklidesowej
def draw_normal(a, b):
  basic_curve = pow(y, 2) - pow(x, 3) - x * a - b
  plt.contour(x.ravel(), y.ravel(), basic_curve, [0]) #tutaj wpisac jaka krzywa do wyrysowania
  plt.grid()
  signb = "" if b < 0 else "+"

# Funkcja rysująca punkty o współrzędnych całkowitych w przestrzeni afinicznej
def draw_afinic(a, b, f):
  with warnings.catch_warnings(): # Pomijanie ostrzeżeń
    warnings.simplefilter("ignore")

    field = ec.SubGroup(f, (0,0), 1, 1) # Generowanie przestrzeni afinicznej
    curve = ec.Curve(a, b, field) # Generowanie krzywej w tej przestrzeni
    coordinates = [] # Kontener na współrzędne

    # Przeszukiwanie współrzędnych całkowitych od 0 do rozmiaru pola za punktami leżącymi na krzywej
    for x in range(f):
      for y in range(f):
        p = ec.Point(curve, x, y)
        if p.on_curve:
          coordinates.append((x, y))

  # Wyrysowanie punktów
  plt.scatter(list(x[0] for x in coordinates), list(x[1] for x in coordinates))

def g(a, b, field):
  plt.figure(figsize=(10, 10)) # Przeskalowanie wykresu
  plt.subplot(211) # Utworzenie pierwszego podwykresu

  # Ustawienie tytułu
  plt.title(f"$y^2 = x^3 {sign_format(a)} \cdot x {sign_format(b)}\ field = {field}$")
  draw_normal(a,b)
  plt.subplot(212) # Utworzenie drugiego podwykresu
  draw_afinic(a, b, field)
  plt.show()

# Interaktywne wyświetlanie wykresu z suwakami
l = interactive(g, a=(-5, 5, 0.1), b=(-5, 5, 0.1), field=(1, 100, 1))
display(l)

interactive(children=(FloatSlider(value=0.0, description='a', max=5.0, min=-5.0), FloatSlider(value=0.0, descr…

W celu graficznego przedstawienia, omówimy poniżej przykład krzywej **$y^2$ = $x^3$ - x + 1**, nazwijmy ją krzywa **E**. Oto jej wykres na płaszczyźnie euklidesowej.


![alt text](https://drive.google.com/uc?id=1wp53ppsVxNT6ojwaEdVoyNjk_63MC_Z5)

Teraz wyświetlmy przedstawienie dokładnie tej samej krzywej, rzuconą na płaszczyznę afiniczną o skończonym polu wartości 97.

![alt text](https://drive.google.com/uc?id=1xjRdktT80IpTezPre-7SfJ5_Uwjjge4F)

Choć na pierwszy rzut oka, może wydawać się, iż są to dwa, zupełnie różne wykresy, to tak naprawdę zmienia się tylko płaszczyzna, na której wyrysowana jest krzywa eliptyczna **E**. Wyobraźmy sobie, że płaszczyzna na której oryginalnie krzywa była wyrysowana, jest niejako zawijana na krawędziach wyznaczających pole skończone (tutaj o wartości 97), gdy się przyjrzymy widać nawet dokładną symetrię wzdłuż osi X. Najlepiej charakter 'zawijania' przestrzeni zobrazuje poniższa grafika. Płaszczyzna widoczna poniżej jest zaś 'wyprostowanym' wykresem, niczym mapa ziemi reprezentująca cały glob na płaskiej powierzchni.

![alt text](https://drive.google.com/uc?id=1LH742jSokRRjNim3tNHMRg27GQaxrgJE)

Wracając do płaszczyzny afinicznej, wszystkie punkty oznaczone kolorem niebieskim przedstawiają miejsca w któych koordynaty krzywej osiągają pełne wartości, np. (70,6),(76,48). Dzięki zastosowaniu płaszczyzny skończonej, możliwe jest wyznaczanie kolejnych punktów dzięki temu samemu schematowi pokazanemu we wcześniejszym schemacie na płaszczyźnie euklidesowej.

![alt text](https://drive.google.com/uc?id=1xsSRUTkBNMI1CeVuO2snJ1c9MckQaKvZ)



Poniżej możemy zobaczyć przykład zbioru punktów afinicznych krzywej **$y^2 = x^3 - x$**, na skończonej płaszczyźnie oznaczonej jako F61 gdzie cyfra oznacza zakres osi X oraz Y.

![alt text](https://drive.google.com/uc?id=1c_W6YZXA2T7SmqjWXuuc2KlMgjRd1dn5)

Dzięki zastosowaniu tego (nieintuicyjnego) sposobu prezentacji krzywych eliptycznych, możemy zaprezentować wiadomość jako zbiór punktów na krzywej. 
Kryptografię krzywych eliptycznych można sprowadzić do wybrania liczby pierwszej jako maksimum, równania krzywej i jakiegoś punktu będącego wartością publiczną. 

Następnie wybieramy kluczy prywatny **R**, będący dużym numerem, oraz klucz publiczny, będący wcześniejszą wartością publiczną pomnożoną przez samą siebie **R** razy. Zadanie i trudność związana z obliczeniem klucza prywatnego z klucza publicznego nazywana jest funkcją dyskretnego logarytmu krzywych eliptycznych. 



## Bezpieczeństwo krzywych eliptycznych

Wykorzystując kwantowy algorytm [Shor'a](https://en.wikipedia.org/wiki/Shor%27s_algorithm), możliwe jest złamanie szyfrowania krzywych eliptycznych wykorzystując kwantowe komputery. Estymuje się, że aby złamać kod o długości 128-bitów wymagane byłoby wykorzystanie komputera posiadającego 2330 bity kwantowe.
Dla porównania złamanie szyfrowania RSA o długości 2048 bitów wymagałoby wykorzystania 4098 bitów kwantowych.
Obecnie, najpotężniejszy komputer kwantowy wyposażony jest w 53 bity, więc złamanie tych szyfrowań za pomocą działań algorytmicznych przez najbliższe dekady nie będzie prawdopodobnie możliwe.

# Najpopularniejsze krzywe elitpyczne

Większość profesjonalnie używanych krzywych eliptycznych, z wiadomych względów, jest utajona oraz objęta patentami, jednak część powszechnie używanych dostępna jest do publicznego badania i wykorzystania. Oto dwie, obecnie najpopularniejsze.

## Krzywa 25519

Jest jedną z najszybszych do obliczenia krzywych, oferuje 128 bitów i została specjalnie zaprojektowana do współpracowania z algorytmem wymiany Diffie-Hellmana(DH). W ostatnich latach, w związku z skandalem w agencji NSA Stanów Zjednoczonych, znacząco zyskała na popularności zastępując używaną dotychczas krzywą P-256. 
W 2014 **25519** stała się standardem wymiany DH w protokole OpenSSH, w 2017 została zaakceptowana jako jedna z krzywych używanych do szyfrowania komunikacji przez instytucje USA.

![alt text](https://drive.google.com/uc?id=1rhrWzNSSwvFkAz4l2ageaprXepnb0XOD)

## Krzywa Secp256k1

Na pierwszy rzut oka niewyróżniająca się, będąca jednak jedną z najważniejszych i najpopularniejszych krzywych na świecie. Ze względu na swoją wysoką wydajność przy obliczeniach oraz bezpieczeństwo, została wybrano jako podstawowa metoda obliczania kluczy podczas wymiany kryptowaluty **Bitcoin**. 

Większość krzywych używanych w kryptografii zawiera w sobie pewien stopień losowości, jednak Secp256k1 została stworzona żeby być przewidywalną, w celu zmniejszenia szansy na zaimplementowanie w niej jakiegoś rodzaju backdoora.


![alt text](https://drive.google.com/uc?id=1IXx7dv62pfQRq7wRaYbc_Unb1n6iLTHC)

Więcej krzywych, wraz z ich opisami można znaleźć na: https://safecurves.cr.yp.to".

W poniższych przykładach zostanie użyta krzywa brainpoolIP384t1.

# **Kod ECC (cryptographic elliptic curve)**

## Generowanie kluczy 


### Generowanie za pomocą biblioteki tinyec oraz secrets

Poniżej generujemy parę kluczy dla odbiorcy przy pomocy kryptografii krzywych eliptycznych (krzywa brainpoolP256r-1), następnie zaś generujemy współdzielony sekretny klucz (do szyfrowania) i klucz publiczny zaszyfrowanego tekstu (dla [ECDH]("https://en.wikipedia.org/wiki/Elliptic-curve_Diffie–Hellman")) z klucza publicznego odbiorcy. Później generujemy ten sam współdzielony sekretny klucz(do deszyfracji) z prywatnego klucza odbiorcy i klucza publicznego tekstu.

Kod ma za zadanie zobrazować, że jesteśmy w stanie wygenerować parę kluczy(sekretny + publiczny wiadomości) za pomocą podanego punktu danej krzywej eliptycznej, zaś później możemy odtworzyć dokładnie ten sam klucz przy pomocy pary kluczy (publiczny wiadomości + prywatny odbiorcy)

In [0]:
## Funckje do komórki niżej

brain_curve = registry.get_curve('brainpoolP256r1') # wczytanie okreslonej krzywej


# kompresja współrzędnych punktu (x,y) do postaci jednej wartości szesnastkowej
def compress_point(point):
    return hex(point.x) + hex(point.y % 2)[2:]

# obliczanie kluczy szyfrujących
def ecc_calc_encryption_keys(pubKey):
    ciphertextPrivKey = secrets.randbelow(brain_curve.field.n) # generowanie klucza prywatnego na podstawie krzywej
    ciphertextPubKey = ciphertextPrivKey * brain_curve.g # klucz publiczny na podstawie prywatnego pomonzonego prze punkt generatora krzywej
    sharedECCKey = pubKey * ciphertextPrivKey # wspoldzielony sekretny klucz
    return (sharedECCKey, ciphertextPubKey)

# obliczanie klucza wspołdzielonego sekretnego klucza
def ecc_calc_decryption_key(privKey, ciphertextPubKey):
    sharedECCKey = ciphertextPubKey * privKey
    return sharedECCKey

In [0]:
privKey = secrets.randbelow(brain_curve.field.n) # analogicznie jak komorke wyzej generujemy klucz prywatny
pubKey = privKey * brain_curve.g 
print("private key:", hex(privKey))
print("public key:", compress_point(pubKey))

(encryptKey, ciphertextPubKey) = ecc_calc_encryption_keys(pubKey) 
print("ciphertext pubKey:", compress_point(ciphertextPubKey))
print("encryption key:", compress_point(encryptKey))

decryptKey = ecc_calc_decryption_key(privKey, ciphertextPubKey)
print("decryption key:", compress_point(decryptKey))

private key: 0x9b8332444d36edb59186120733d8c331ae4c3c12f161d2a8806211219756ab2e
public key: 0x6d4a5482940dd493385d2c4d56bd815ef57280528c03ea334f9da6eba024191a0
ciphertext pubKey: 0x2b5a75049bed38f7d9f6f4eb2c4bf2e6e6127e119d65dd509afc3afe632840a91
encryption key: 0x349d1863a77c5339f2a3c334a57d98b65515e61a7884c968658d0dd9ebabe9800
decryption key: 0x349d1863a77c5339f2a3c334a57d98b65515e61a7884c968658d0dd9ebabe9800


Należy zaznaczyć fakt, że biblioteka tinyec nie nadaje się do użycia produkcyjnego. Jest to biblioteka wyłącznie edukacyjna. Poniżej podane są przykłady bibliotek, które można wykorzystać w projektach. 

### Generowanie za pomocą biblioteki fastedcsa

In [0]:
priv_key = keys.gen_private_key(curve.secp256k1)
pub_key = keys.get_public_key(priv_key, curve.secp256k1)
print(f"Klucz publiczny krzywej: {pub_key}\nKluczy prywatny: {priv_key}")

Klucz publiczny krzywej: X: 0x737fc58238878aad7977c1b9a3ddec7f4880f3c2cb9dbc7373c2093577f23838
Y: 0x274f241ef9bc94f73673cc14a47507b2a09e0b5d52c1e5a20cb89f1c0d1f2b98
(On curve <secp256k1>)
Kluczy prywatny: 5690907813090208776787406677933837151611769834352404146188327869891793290324


In [0]:
messeage = b"Ala ma kota"

#domyslne szyfrowanie biblioteki fastecdsa - P256 i SHA2
priv_key = keys.gen_private_key(curve.P256)
pub_key = keys.get_public_key(priv_key, curve.P256)

# podpisywanie wiadomości, zwraca 2 wartości całkowitoliczbowe
r,s = ecdsa.sign(messeage, priv_key)
# walidacja poprawności sygnatury klucza, powinno zwrócić True
valid = ecdsa.verify((r,s), messeage, pub_key)
print(f"Sygnatura jest poprawana: {valid}")


Sygnatura jest poprawana: True


## Zapis i odczyt tworzonych kluczy z/do pliku

In [0]:
priv, public = keys.gen_keypair(curve.secp256k1) # od razu generuje publiczny i prywatny klucz

keys.export_key(priv, curve=curve.secp256k1, filepath='/secp256k1.key') # zapisze na dysku maszyny colab!
keys.export_key(public,curve=curve.secp256k1, filepath='/secp256k1.pub')

## odczyt z pliku do zmiennych:
# wczytujemy do dwóch zmiennych, jeśli odczytujemy klucz prywatny to pierwsza będzie wartością klucza, 
# drugą będzie wartościami x/y określającymi miejsce tej wartości na krzywej(kluczem publicznym)
parsed_priv, point = keys.import_key('/secp256k1.key')
print(f"Wczytany klucz prywatny to {parsed_priv} \nNatomiast wartości tego punktu na krzywej wynoszą:\n{point}")

# Jeśli odczytujemy tylko klucz publiczny, pierwsza wartość wyniesie None, zapewnia to dodatkowe bezpieczeństwo
parsed_priv, point = keys.import_key('/secp256k1.pub')
print(f"\n\nWczytany klucz prywatny to {parsed_priv} \nNatomiast wartości tego punktu na krzywej wynoszą:\n{point}")


Wczytany klucz prywatny to 26050088249531758203048464788284957184462680351141457887272198764383414269746 
Natomiast wartości tego punktu na krzywej wynoszą:
X: 0xc9ef028dec722cb7ca648c3b45dc59f976ef96bb39eed37d149ff6838d4b5805
Y: 0xaa107eb23eb736709c0edf073a3b394b2eb5e35def0eeea32c0b8cbe0ed16338
(On curve <secp256k1>)


Wczytany klucz prywatny to None 
Natomiast wartości tego punktu na krzywej wynoszą:
X: 0xc9ef028dec722cb7ca648c3b45dc59f976ef96bb39eed37d149ff6838d4b5805
Y: 0xaa107eb23eb736709c0edf073a3b394b2eb5e35def0eeea32c0b8cbe0ed16338
(On curve <secp256k1>)


### Generowanie kluczy za pomocą biblioteki ecpy

In [0]:
from ecpy.curves     import Curve,Point
from ecpy.keys       import ECPublicKey, ECPrivateKey
from ecpy.ecdsa      import ECDSA

cv   = Curve.get_curve('secp256k1')
pu_key = ECPublicKey(Point(0x65d5b8bf9ab1801c9f168d4815994ad35f1dcb6ae6c7a1a303966b677b813b00,
                       0xe6b865e529b8ecbf71cf966e900477d49ced5846d7662dd2dd11ccd55c0aff7f,
                       cv))
pv_key = ECPrivateKey(0xfb26a4e75eec75544c0f44e937dcf5ee6355c7176600b9688c667e5c283b43c5,
                  cv)


signer = ECDSA()
sig    = signer.sign(b'01234567890123456789012345678912',pv_key)
assert(signer.verify(b'01234567890123456789012345678912',sig,pu_key))

## Szyfrowanie oraz deszyfrowanie wiadomości - biblioteki tinyec, secrets, haslib, Cipher

Poniżej przedstawiono pełny kod hybrydowego szyfrowania [ECC](https://en.wikipedia.org/wiki/Elliptic-curve_cryptography)/[AES]("https://pl.wikipedia.org/wiki/Advanced_Encryption_Standard"), Schemat działania wygląda następująco:
- Początkowo generujemy parę kluczy ECC dla odbiorcy wiadomości, zostaną one użyte do hybrydowego(asymetryczne ECC i symetryczne AES) zaszyfrowania wiadomości msg i późniejszego jej odszyfrowania. 
- Następnie szyfrujemy **msg** za pomocą **pubKey**, i w rezultacie otrzymujemy zmienne **ciphertext, nonce, authTag, cipgertextPubKey** jako output.
  - ciphertext: zaszyfrowany tekst przez symetryczny losowy wektor AES.
  - authTag: kod MAC zaszyfrowanego tekstu
  - ciphertextPubKey: losowo wygenerowany klucz publiczny tekstu wysyłany razem z wiadomością, wykorzystywany do wygenerowania klucza AES przez odbiorcę.
- funkcja **encrypt_ECC(msg, pubKey)** generuje parę kluczy ECC oraz oblicza szyfrowanie klucza współdzielonego **sharedECCKey**, który jest punktem na krzywej eliptycznej. Jest on następnie zmieniany na 256-bitowy klucz AES poprzez haszowanie jego punktów x i y do jednej zmiennej.
- Odszyfrowanie wiadomości przebiega w odwróconej kolejności, najpierw obliczamy współdzielony symetryczny klucz ECC, następnie jest on również haszowany do 256 bitów, kolejnym krokiem jest użycie szyfru **AES-256-GCN** do odszyfrowania tekstu składającego się z **ciphertext + nonce + authTag** dzięki współdzielonemu sekretnemu kluczu.
- Wynikiem tych skomplikowanych operacji jest otrzymanie zwykłego tekstu zawierającego nadaną wiadomość.

In [0]:
# funkcję do komórki niżej

def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM) # wygenerowanie szyfrowania AES
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg) # zaszyfrowanie wiadomości
    return (ciphertext, aesCipher.nonce, authTag) 

def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce) # tym razem z dodatkowym argumentem żeby pokrywał się z pierwszym szyfrowaniem
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big')) # hashowanie przesłanego punktu do 256-bitowego klucza
    sha.update(int.to_bytes(point.y, 32, 'big')) # zmniana z int na bajty
    return sha.digest()
    
def encrypt_ECC(msg, pubKey): # szyfrowanie asymetryczne z użyciem krzywych eliptycznych
    ciphertextPrivKey = secrets.randbelow(curve.field.n) # generowanie klucza prywatnego jako wartości punktu z krzywej eliptycznej
    sharedECCKey = ciphertextPrivKey * pubKey # wyliczenie klucza publicznego
    secretKey = ecc_point_to_256_bit_key(sharedECCKey) # wywołanie funkcji hashującej klucz prywatny
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey) # wywołanie funkcji szyfrującej wiadomość 
    ciphertextPubKey = ciphertextPrivKey * curve.g # stworzenie klucza publicznego zaszyfrowanej wiadomości
    return (ciphertext, nonce, authTag, ciphertextPubKey)

def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg # w zasadzie odwrócenie kolejności działań powyższej funkcji
    sharedECCKey = privKey * ciphertextPubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

In [0]:
curve = registry.get_curve('brainpoolP256r1') # wczytujemy wzór odpowiedniej krzywej eliptycznej

msg = b'Ala ma kota' # wiadomość, litera 'b' oznacza zapisanie do zmiennej typu bytes zamiast typowego stringa
print("oryginalna wiadomość:", msg)
privKey = secrets.randbelow(curve.field.n) # wartość klucza prywatnego pobieramy jako losową_poniżej(krzywa.wielkość_płaszczyzny.punkty)
pubKey = privKey * curve.g # obliczanie klucza publicznego

encryptedMsg = encrypt_ECC(msg, pubKey) # wywołanie funkcji z komórki powyżej
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]), # ta i poniższe komendy mają za zadanie zwrócić szesnastkową wersję danych binarnych
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
}
print("zaszyfrowana wiadomość:", encryptedMsgObj) 

decryptedMsg = decrypt_ECC(encryptedMsg, privKey) # wywołanie funkcji z komórki powyżej
print("poprawnie odszyfrowana wiadomość:", decryptedMsg)


oryginalna wiadomość: b'Ala ma kota'
zaszyfrowana wiadomość: {'ciphertext': b'3d1541b7718a33d1b26ab8', 'nonce': b'60075a443c03c7354f14159f26a97cf4', 'authTag': b'd60c4aa770d1f9a6912b0c5b4475c86c', 'ciphertextPubKey': '0x370ea84aa9432215107e17ad8d8a84a328c11a8ffb3d17c609650551bbaf0f161'}
poprawnie odszyfrowana wiadomość: b'Ala ma kota'


# Materiały do dalszej nauki

https://asecuritysite.com/encryption/ecdh3 - na tej stronie możemy szybko sprawdzić przykłady szyfrowania wraz z wypisaniem wygenerowanych wartości, dla wielu najpopularniejszych krzywych

# Konwerter do formatu PDF

In [0]:
# Należy czytać output,  w pewnym momencie poprosi o wejście w podany link i skopiowanie kodu autoryzacji żeby móc zapisać pdf na swoim dysku drive
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc
from google.colab import drive
drive.mount('/content/drive')
!cp drive/My Drive/Colab Notebooks/krzywe_eliptyczne.ipynb ./
!pwd
!jupyter nbconvert --to PDF "krzywe_eliptyczne.ipynb"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
texlive is already the newest version (2017.20180305-1).
texlive-latex-extra is already the newest version (2017.20180305-2).
texlive-xetex is already the newest version (2017.20180305-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat 'drive/My': No such file or directory
cp: cannot stat 'Drive/Colab': No such file or directory
cp: cannot stat 'Notebooks/krzywe_eliptyczne.ipynb': No such file or directory
/content


# **Źródła**


- Elliptic Curves - Computerphile: https://www.youtube.com/watch?v=NF1pwjL9-DE
- Elliptic Curve Cryptography Tutorial - Understanding ECC through the Diffie-Hellman Key Exchange - https://www.youtube.com/watch?v=gAtBM06xwaw
- Krzywe eliptyczne PL - https://pl.wikipedia.org/wiki/Krzywa_eliptyczna
- wstęp do krzywych eliptycznych - https://qvault.io/2019/12/31/very-basic-intro-to-elliptic-curve-cryptography/
- Linia czasu komputerów kwantowych - https://en.wikipedia.org/wiki/Timeline_of_quantum_computing
- Krzywe eliptyczne - https://en.wikipedia.org/wiki/Elliptic-curve_cryptography
- Algorytm Shor'a - https://en.wikipedia.org/wiki/Shor%27s_algorithm
- Krzywa 25519 - https://en.wikipedia.org/wiki/Curve25519
- secp256k1 https://en.bitcoin.it/wiki/Secp256k1
- kryptografia EC - https://cryptobook.nakov.com/asymmetric-key-ciphers/ecc-encryption-decryption
- post odnośnie krzywych na płaszczyznach skończonych - https://crypto.stackexchange.com/questions/66589/elliptic-curves-on-finite-fields
- algorytm Schoof'a - https://en.wikipedia.org/wiki/Schoof%27s_algorithm
- Płaszczyzna afiniczna - https://en.wikipedia.org/wiki/Affine_space
- obliczenia na płaszczyznach skończoncych - https://andrea.corbellini.name/2015/05/23/elliptic-curve-cryptography-finite-fields-and-discrete-logarithms/
- protokół wymiany ECDH - https://www.youtube.com/watch?v=F3zzNa42-tQ
- streszczenie kryptografii EC - https://www.youtube.com/watch?v=dCvB-mhkT0w
- Dodatkowy opis działań na płaszczyznach afinicznych ECC - https://fangpenlin.com/posts/2019/10/07/elliptic-curve-cryptography-explained/
- AES - https://pl.wikipedia.org/wiki/Advanced_Encryption_Standard
- ECC - https://cryptobook.nakov.com/asymmetric-key-ciphers/ecc-encryption-decryption
